In [1]:
import csv
import pandas as pd
import numpy as np
import glob
import matplotlib
import scipy as sp
from matplotlib import pyplot as plt
from itertools import cycle, islice
from scipy import stats
from datetime import datetime

import seaborn as sns

In [2]:
'''This notebook imports all of the SMAB3 and SMAB4 datasets and puts them into dataframes. 
The dataframes are stored in dictionaries that can be accessed using 

%store -r variable

in another notebook in this repository.
For example, to use the SMAB3 control data dictionary called 's_dict' in another notebook, run the line:

%store -r s_dict'''

"This notebook imports all of the SMAB3 and SMAB4 datasets and puts them into dataframes. \nThe dataframes are stored in dictionaries that can be accessed using \n\n%store -r variable\n\nin another notebook in this repository.\nFor example, to use the SMAB3 control data dictionary called 's_dict' in another notebook, run the line:\n\n%store -r s_dict"

In [3]:
'''The variables from this notebook available to use in other notebooks:
s_dict
c_dict
m_dict
j_dict

where 's' is SMAB3 control, 'c' is SMAB4 control, 'm' is SMAB4 social, 'j' is SMAB3 social '''

"The variables from this notebook available to use in other notebooks:\ns_dict\nc_dict\nm_dict\nj_dict\n\nwhere 's' is SMAB3 control, 'c' is SMAM4 control, 'm' is SMAB4 social, 'j' is SMAB3 social "

In [4]:
#NEED TO: remove bad sessions from dictionairies 
#organize sessions into folders based on type and then automatically read all files in folder 

# Functions

In [5]:
# function to return key for any value
def get_key(my_dict, val):
    for key, value in my_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [83]:
'''
Here, we load all of the csv files into pandas dataframes using organize_trial_by_trial and get the game 1 and 2 scores using 
call_get_score.

This function is called separately for each group of data (SMAB3 control, SMAB3 social, SMAB4 control, SMAB4 social)'''
def imports(sessions,condition,types='default'):
    #create dictionary
    dicti = {}
    
    #loop through the list of sessions
    for i in sessions:
        
        print(i)
        
        #set path based on session
        path = 'data/' + i + '_gamedata.csv'

        #call function to make dataframe
        df = organize_trial_by_trial(path,types)
        
        #add dataframe to dictionary
        dicti[i] = df

        
        if condition == 's' or condition == 'c':
            
            #call get score function and add scores to dictionary, with the relevant keys
            dicti[i + '_p1_g1_score'],dicti[i + '_p1_g2_score'],dicti[i + '_score_diff'],dicti[i + '_p1_g1_50'],dicti[i + '_p1_g2_50'] = call_get_score(df, condition)
        
            #call separate_df function to separate the df's by player and game numbers; add df's to dictionary, with the relevant keys
            dicti[i + '_p1_g1'], dicti[i + '_p1_g2'] = separate_df(df, condition)
        
        
            #call fix_reward for frames where type is 'three' and condition is 'c' or 'm'
            
            if condition == 'c':
                if types == 'three':
                    dicti[i + '_p1_g1'] = fix_reward(dicti[i + '_p1_g1'])
                    dicti[i + '_p1_g2'] = fix_reward(dicti[i + '_p1_g2'])
                    
                    
                    
        elif condition == 'm' or condition == 'j':
            
            #call get score function and add scores to dictionary, with the relevant keys
            dicti[i + '_p1_g1_score'],dicti[i + '_p1_g2_score'],dicti[i + '_p2_g1_score'],dicti[i + '_p2_g2_score'],dicti[i + '_p1_diff'],dicti[i + '_p2_diff'],dicti[i + '_p1_g1_50'],dicti[i + '_p1_g2_50'],dicti[i + '_p2_g1_50'],dicti[i + '_p2_g2_50']= call_get_score(df, condition)
         
            #call separate_df function to separate the df's by player and game numbers; add df's to dictionary, with the relevant keys
            dicti[i + '_p1_g1'], dicti[i + '_p1_g2'], dicti[i + '_p2_g1'], dicti[i + '_p2_g2'] = separate_df(df, condition)
        
            #call fix_reward for frames where type is 'three' and condition is 'c' or 'm'
            
            '''
            if condition == 'm':
                if types == 'three':
                    dicti[i + 'p1_g1'] = fix_reward(dicti[i + 'p1_g1'])
                    dicti[i + 'p1_g2'] = fix_reward(dicti[i + 'p1_g2'])
                    dicti[i + 'p2_g1'] = fix_reward(dicti[i + 'p2_g1'])
                    dicti[i + 'p2_g2'] = fix_reward(dicti[i + 'p2_g2'])'''
    
    return dicti

In [7]:
'''
These functions import csv files into pandas dataframes and rename the columns.
input: filename of the csv file to import; the path needs to be included, as well
output: pandas dataframe

There are multiple iterations of this function because the data is in slightly different formats. 

TO DO: convert this to 1 function.
'''


def organize_trial_by_trial(txt_filename, types):
    reader = csv.reader( open(txt_filename))
    game_data = [row for row in reader]
    df = pd.DataFrame(np.array(game_data))
    df = df.T
    if types == 'default':
        df = df.rename(columns = {0:"game number", 1:"trial", 2:"player", 3:"arm", 4:"probability", 5:"reward", 6:"time", 7:"P1 score", 8:"P2 score"})
    elif types == 'three':
        df = df.rename(columns = {0:"type", 1:"game number", 2:"trial", 3:"player", 4:"arm", 5:"probability", 6:"reward", 7:"time", 8:"P1 score", 9:"P2 score"})
    elif types == 'floatclick':
        df = df.rename(columns = {0:"type", 1:"game number", 2:"trial", 3:"player", 4:"arm", 5:"probability", 6:"P1 score", 7:"P2 score", 8:""})
    #drops first 20 trials of game 1
    #remove if you want to analyze the whole session
    #df = df[40:]
    #df = df.drop(columns = 'time') #can comment this out if want to look at time
    #df = df.drop(['reward'], axis=1)
    df = df.apply(pd.to_numeric)
    df.head()
    
    return df


#used before changes were made to template
def organize_trial_by_trial_hand_code(txt_filename):
    reader = csv.reader( open(txt_filename))
    game_data = [row for row in reader]
    df = pd.DataFrame(np.array(game_data))
    #remove first row with column headings
    df = df.drop(df.index[0])
    #convert values to numbers (NEED TO FIX)
    df.reset_index(drop=True,inplace=True)
    df = df.drop(range(0, 8))
    df.reset_index(drop=True,inplace=True)
    #df = df.apply(pd.to_numeric)
    #add column headings
    df = df.rename(columns = {0:"game number", 1:"trial", 2:"player", 3:"arm", 4:"probability", 5:"reward",6:"time",7:"P1 score", 8:"P2 score"})
    df.head()
    
    return df

#use this function for hand coded game data files 
def organize_trial_by_trial_hand_code2(txt_filename):
    reader = csv.reader( open(txt_filename))
    game_data = [row for row in reader]
    df = pd.DataFrame(np.array(game_data))
    #remove first row with column headings
    df = df.drop(df.index[0])
    #convert values to numbers (NEED TO FIX)
    df.reset_index(drop=True,inplace=True)
    df = df.drop(range(0, 8))
    df.reset_index(drop=True,inplace=True)
    #drop comment columns
    #df = df.drop(range(7,9), axis = 1)
    #add p1 and p2 score columns
    df.insert
    #add column headings
    df = df.rename(columns = {0:"game number", 1:"trial", 2:"player", 3:"arm", 4:"probability", 5:"reward",6:"time",7:"comment 1", 8:"comment 2", 9:"P1 score", 10:"P2 score"})
    
    #remove the time and comment columns (we'll add them back in a moment)
    #we are removing them bc we are converting all the values to numbeic and this can't be done to these columns
    time = df['time']
    df = df.drop('time', axis = 1 )
    
    comment_1 = df['comment 1']
    comment_2 = df['comment 2']
    df = df.drop('comment 1', axis = 1 )
    df = df.drop('comment 2', axis = 1 )
    
    df = df.apply(pd.to_numeric)
    
    #add dropped columns back in
    df['time'] = time
    df['comment 1'] = comment_1
    df['comment 2'] = comment_2
    
    #convert time and comment 1 to date time objects
    df['time']= pd.to_datetime(df['time'], errors = 'coerce', format = '%M:%S.%f')
    df['comment 1']= pd.to_datetime(df['comment 1'], errors = 'coerce', format = '%M:%S.%f')

    df.head()
    
    return df

In [8]:
'''
This function returns the score for game 1 and game 2. call_get_score can be used to call this function.
'''
def get_score(df, player_number, game_number):
    if game_number == 1:
        row = 199
    elif game_number == 2:
        row = 399
    if player_number == 1:
        column = 'P1 score'
    elif player_number == 2:
        column = 'P2 score'
    score = df.loc[row, column]
    return score

'''Here we get the score at trial 50 of both games'''
def get_score_at_50(df, player_number, game_number):
    if game_number == 1:
        row = 99
    elif game_number == 2:
        row = 300
    if player_number == 1:
        column = 'P1 score'
    elif player_number == 2:
        column = 'P2 score'
    score = df.loc[row, column]
    return score

In [9]:
'''This function calls get_score.
input: dataframe with game data and version*
output: scores for each player and each game**
*version is 's' for single player SMAB3, 'c' for single player SMAB4, 'j' for multiplayer SMAB3, 
'm' for multiplayer SMAB4
**only 1 player score is outputted for each game in the single player sessions, while scores for both players are 
outputted for multiplayer sessions'''
def call_get_score(df, version):
    p1_g1 = get_score(df, 1, 1)
    p1_g2 = get_score(df, 1, 2)
    p1_score_diff = p1_g2 - p1_g1
    
    p1_g1_50 = get_score_at_50(df, 1, 1)
    p1_g2_50 = get_score_at_50(df, 1, 2)
    
    if version == 'j' or version == 'm':
        p1_g1 = get_score(df, 1, 1)
        p1_g2 = get_score(df, 1, 2)
        p1_score_diff = p1_g2 - p1_g1
        
        p1_g1_50 = get_score_at_50(df, 1, 1)
        p1_g2_50 = get_score_at_50(df, 1, 2)
        
        p2_g1 = get_score(df, 2, 1)
        p2_g2 = get_score(df, 2, 2)
        p2_score_diff = p2_g2 - p2_g1
        
        p2_g1_50 = get_score_at_50(df, 2, 1)
        p2_g2_50 = get_score_at_50(df, 2, 2)
        return p1_g1, p1_g2, p2_g1, p2_g2, p1_score_diff, p2_score_diff, p1_g1_50,p1_g2_50, p2_g1_50, p2_g2_50

    return p1_g1, p1_g2, p1_score_diff, p1_g1_50, p1_g2_50

In [10]:
def remove_first_n_trial(df, n):
    return df.drop(df.head(n).index,inplace=True)

In [11]:
'''This functions separates the dataframes by player and game.
input: dataframe with gamedata and version*
output: four dataframes separated by player and game
*same as call_get_score above'''
def separate_df(df, version):
    df_p1 = df[df['player'] == 1]
    df_p1_g1 = df_p1[df['game number'] == 1]
    df_p1_g2 = df_p1[df['game number'] == 2]
    
    #reset index
    df_p1_g1 = df_p1_g1.reset_index(drop=True)
    df_p1_g2 = df_p1_g2.reset_index(drop=True)
    
    #remove 1st 20 trials
    #df_p1_g1.drop(df_p1_g1.head(20).index,inplace=True)
    #df_p1_g2.drop(df_p1_g2.head(20).index,inplace=True)
    
    if version == 's':
        return df_p1_g1, df_p1_g2
    if version == 'c':
        return df_p1_g1, df_p1_g2
    df_p2 = df[df['player'] ==2 ]
    df_p2_g1 = df_p2[df['game number'] == 1]
    df_p2_g2 = df_p2[df['game number'] == 2]
    
    #reset index
    df_p1_g2.reset_index(drop=True)
    df_p2_g2.reset_index(drop=True)
    
    #remove 1st 20 trials
    #df_p2_g1.drop(df_p2_g1.head(20).index,inplace=True)
    #df_p2_g2.drop(df_p2_g2.head(20).index,inplace=True)
    
    return df_p1_g1, df_p1_g2, df_p2_g1, df_p2_g2

In [12]:
'''This functionfixes the reward column in the files imported using the '300' function. 
This function works for dataframes that have already been split up by game and player by separate_df (ex. c12_p1_g1)
NOTE that this function had to be written for a subset of the data that had a bug that caused the reward column to be
0 for all trials.'''
def fix_reward(df):
    df.reset_index(drop=True, inplace=True)
    for index, row in df.iterrows():
        if df.loc[index, 'player'] == 1:
                score_column = 'P1 score'
        elif df.loc[index, 'player'] == 2:
                score_column = 'P2 score'
        if index == 0:
            if df.loc[index, score_column] == 1:
                df.loc[index, 'reward'] = 1
            #not necessary here because the values are all 0
            else:
                df.loc[index, 'reward'] = 0
        elif index > 0:
            prev_score = df.loc[index - 1, score_column]
            curr_score = df.loc[index, score_column]
            score_diff = curr_score - prev_score
            df.loc[index, 'reward'] = score_diff
    return(df)

In [13]:
'''This function outputs the hole-probability mappings in each game. 
input: df of session data
output: df with the hole-probability mappings
g = game
p = player
h = hole
r = reward (1 means yes and 0 means no)
'''

def hole_prob_map(df):
    h1_prob = df.loc[df['arm'] == 1, 'probability'].iloc[0]
    h2_prob = df.loc[df['arm'] == 2, 'probability'].iloc[0]
    h3_prob = df.loc[df['arm'] == 3, 'probability'].iloc[0]
    h_probs = pd.DataFrame({'h1_prob': [h1_prob], 'h2_prob': [h2_prob], 'h3_prob': [h3_prob]})
    
    return h_probs


# Hand Coding Functions
These functions can be used for game data files that were created by hand coding.

In [14]:
'''This function calculates the number of times each hole was chosen and the 
probability with which each hole rewarded each player in each game.
input: hand coded game data file in a df
output: df with the number of times each hole was chosen by each player in each game and the percentage of choices that were
rewarded
g = game
p = player
h = hole
r = reward (1 means yes and 0 means no)'''

def handcode_calc_prob(df):
    
    #NOTE: turn into function with parameters for game and player
    #get counts for g1, p1
    g1_p1_h1_r1_count,g1_p1_h1_r0_count,g1_p1_h2_r1_count,g1_p1_h2_r0_count,g1_p1_h3_r1_count,g1_p1_h3_r0_count = get_count_hand(df, 1,1)
    
    g1_p2_h1_r1_count,g1_p2_h1_r0_count,g1_p2_h2_r1_count,g1_p2_h2_r0_count,g1_p2_h3_r1_count,g1_p2_h3_r0_count = get_count_hand(df, 2,1)
    
    g2_p1_h1_r1_count,g2_p1_h1_r0_count,g2_p1_h2_r1_count,g2_p1_h2_r0_count,g2_p1_h3_r1_count,g2_p1_h3_r0_count = get_count_hand(df, 1,2)
    
    g2_p2_h1_r1_count,g2_p2_h1_r0_count,g2_p2_h2_r1_count,g2_p2_h2_r0_count,g2_p2_h3_r1_count,g2_p2_h3_r0_count = get_count_hand(df, 2,2)
    
    #calculate reward rate of each hole in each condition
    
    g1_p1_h1 = calc_prob(g1_p1_h1_r1_count,g1_p1_h1_r0_count)
    g1_p1_h2 = calc_prob(g1_p1_h2_r1_count,g1_p1_h2_r0_count)
    g1_p1_h3 = calc_prob(g1_p1_h3_r1_count,g1_p1_h3_r0_count)
    
    g1_p2_h1 = calc_prob(g1_p2_h1_r1_count,g1_p2_h1_r0_count)
    g1_p2_h2 = calc_prob(g1_p2_h2_r1_count,g1_p2_h2_r0_count)
    g1_p2_h3 = calc_prob(g1_p2_h3_r1_count,g1_p2_h3_r0_count)
    
    g2_p1_h1 = calc_prob(g2_p1_h1_r1_count,g2_p1_h1_r0_count)
    g2_p1_h2 = calc_prob(g2_p1_h2_r1_count,g2_p1_h2_r0_count)
    g2_p1_h3 = calc_prob(g2_p1_h3_r1_count,g2_p1_h3_r0_count)
    
    g2_p2_h1 = calc_prob(g2_p2_h1_r1_count,g2_p2_h1_r0_count)
    g2_p2_h2 = calc_prob(g2_p2_h2_r1_count,g2_p2_h2_r0_count)
    g2_p2_h3 = calc_prob(g2_p2_h3_r1_count,g2_p2_h3_r0_count)
    
    
    #NEED to add indices to these
    p1_counts_d = {'g1_p1_h1_count': [g1_p1_h1_r1_count, g1_p1_h1_r0_count], 'g1_p1_h2_count': [g1_p1_h2_r1_count, g1_p1_h2_r0_count],
        'g1_p1_h3_count': [g1_p1_h3_r1_count, g1_p1_h3_r0_count], 'g2_p1_h1_count': [g2_p1_h1_r1_count, g2_p1_h1_r0_count], 
         'g2_p1_h2_count': [g2_p1_h2_r1_count, g2_p1_h2_r0_count],
        'g2_p1_h3_count': [g2_p1_h3_r1_count, g2_p1_h3_r0_count] }
    p1_counts_df = pd.DataFrame(p1_counts_d)
    
    p2_counts_d = {'g1_p2_h1_count': [g1_p2_h1_r1_count, g1_p2_h1_r0_count], 'g1_p2_h2_count': [g1_p2_h2_r1_count, g1_p2_h2_r0_count],
        'g1_p2_h3_count': [g1_p2_h3_r1_count, g1_p2_h3_r0_count], 'g2_p2_h1_count': [g2_p2_h1_r1_count, g2_p2_h1_r0_count], 
         'g2_p2_h2_count': [g2_p2_h2_r1_count, g2_p2_h2_r0_count],
        'g2_p2_h3_count': [g2_p2_h3_r1_count, g2_p2_h3_r0_count] }
    p2_counts_df = pd.DataFrame(p2_counts_d)
    
    p1_counts_df.index = ['rewarded', 'non-rewarded']
    p2_counts_df.index = ['rewarded','non-rewarded']
    
    p1_probs_d = {'p1_h1':[g1_p1_h1, g2_p1_h1], 'p1_h2':[g1_p1_h2, g2_p1_h2], 'p1_h3':[g1_p1_h3, g2_p1_h3]}
    p1_probs_df = pd.DataFrame(p1_probs_d)
    
    p2_probs_d = {'p2_h1':[g1_p2_h1, g2_p2_h1], 'p1_h2':[g1_p2_h2, g2_p2_h2], 'p1_h3':[g1_p2_h3, g2_p2_h3]}
    p2_probs_df = pd.DataFrame(p2_probs_d)
    
    p1_probs_df.index = ['game 1', 'game 2'] 
    p2_probs_df.index = ['game 1','game 2']
    
    return p1_counts_df, p2_counts_df, p1_probs_df, p2_probs_df


In [15]:
'''This function is called in handcode_calc_prob and calculates the number of times each hole was chosen and rewarded/not rewarded
input: df, player and game numbers
output: counts for each hole and outcome (rewarded/not rewarded)
'''
def get_count_hand(df,player,game):
    h1_r1_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==1) & (df['reward']==1)])
    h1_r0_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==1) & (df['reward']==0)])
    h2_r1_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==2) & (df['reward']==1)])
    h2_r0_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==2) & (df['reward']==0)])
    h3_r1_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==3) & (df['reward']==1)])
    h3_r0_count = len(df[(df['game number']==game) & (df['player']==player) & (df['arm']==3) & (df['reward']==0)])
    return h1_r1_count, h1_r0_count, h2_r1_count, h2_r0_count, h3_r1_count, h2_r0_count

In [16]:
'''This function is called in handcode_calc_prob and calculates the reward probability of a hole.
input: number of rewarded trials, number of un-rewarded trials
output: percentage of rewarded trials for that hole'''
def calc_prob(r1,r0):
    try:
        return (r1/(r1+r0) * 100)
    #if the hole was never chosen and the denominator is 0
    except ZeroDivisionError:
        return 0

In [17]:
'''This function fills in the dataframe with the probabilities (30, 60, or 90) to each hole for each player in each
game.
input: dataframe with gamedata, probabilities for each player/game number/hole*
output: dataframe with the probabilities filled in
*You must put the inputs in the correct order as defined in the parameters of the function.
'''
def fill_in_prob(df, p1_g1_h1, p1_g1_h2, p1_g1_h3, p1_g2_h1, p1_g2_h2, p1_g2_h3, p2_g1_h1, p2_g1_h2, p2_g1_h3, 
                 p2_g2_h1, p2_g2_h2, p2_g2_h3):
    for index, row in df.iterrows():
        if df.loc[index,'game number'] == 1:
            if df.loc[index,'player'] == 1:
                if df.loc[index,'arm'] == 1:
                    df.loc[index,'probability'] = p1_g1_h1
                elif df.loc[index,'arm'] == 2:
                    df.loc[index,'probability'] = p1_g1_h2
                elif df.loc[index,'arm'] == 3:
                    df.loc[index,'probability'] = p1_g1_h3
            elif df.loc[index,'player'] == 2:
                if df.loc[index,'arm'] == 1:
                    df.loc[index,'probability'] = p2_g1_h1
                elif df.loc[index,'arm'] == 2:
                    df.loc[index,'probability'] = p2_g1_h2
                elif df.loc[index,'arm'] == 3:
                    df.loc[index,'probability'] = p2_g1_h3
        elif df.loc[index,'game number'] == 2:
            if df.loc[index,'player'] == 1:
                if df.loc[index,'arm'] == 1:
                    df.loc[index,'probability'] = p1_g2_h1
                elif df.loc[index,'arm'] == 2:
                    df.loc[index,'probability'] = p1_g2_h2
                elif df.loc[index,'arm'] == 3:
                    df.loc[index,'probability'] = p1_g2_h3
            elif df.loc[index,'player'] == 2:
                if df.loc[index,'arm'] == 1:
                    df.loc[index,'probability'] = p2_g2_h1
                elif df.loc[index,'arm'] == 2:
                    df.loc[index,'probability'] = p2_g2_h2
                elif df.loc[index,'arm'] == 3:
                    df.loc[index,'probability'] = p2_g2_h3
    return df

# Import Data

# SMAB3 Control Data

In [18]:
#trial_by_trial, three, and floatclick refer to the format of the csv files, which differ slightly
#TO DO: move these files into folders and import using function based on folder
s_trial_by_trial = ['s01','s03','s05','s06','s07','s08','s09']
s_three = ['s11','s12','s15','s16','s17','s19']
s_floatclick = ['s20']

In [19]:
#call imports function for these lists of sessions
s_trial_by_trial_dict = imports(sessions = s_trial_by_trial, condition = 's')

s01
s03
s05
s06
s07
s08
s09


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anacon

In [20]:
s_three_dict = imports(sessions = s_three,condition = 's', types = 'three')

s11
s12
s15
s16
s17
s19


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anacon

In [21]:
s_floatclick_dict = imports(sessions = s_floatclick, condition = 's', types = 'floatclick')

s20


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [22]:
#combine dictionaries into one
s_dict = {**s_trial_by_trial_dict, **s_three_dict}
s_dict

{'s01':      game number  trial  player  arm  probability  reward       time  \
 0              1      1       1    2           30       1  11.220698   
 1              1      1       2    1           90       1   2.742137   
 2              1      2       1    3           60       1   5.362458   
 3              1      2       2    1           90       1   3.699948   
 4              1      3       1    2           30       0   3.489552   
 ..           ...    ...     ...  ...          ...     ...        ...   
 395            2     98       2    1           90       1   2.578115   
 396            2     99       1    1           60       1   2.876227   
 397            2     99       2    1           90       1   2.675604   
 398            2    100       1    1           60       1   2.678057   
 399            2    100       2    1           90       1   2.680739   
 
      P1 score  P2 score  
 0           1         0  
 1           1         1  
 2           2         1  
 3     

In [23]:
s_dict['s11_p1_g1']

,type,game number,trial,player,arm,probability,reward,time,P1 score,P2 score
0,300,1,1,1,3,90,1,17.304740,1,0
1,300,1,2,1,1,30,0,3.604281,1,1
2,300,1,3,1,2,60,1,2.629745,2,2
3,300,1,4,1,1,30,0,2.646737,2,3
4,300,1,5,1,3,90,1,2.468822,3,4
...,...,...,...,...,...,...,...,...,...,...
95,300,1,95,1,1,30,0,1.409328,58,86
96,300,1,97,1,2,60,1,3.733218,59,87
97,300,1,98,1,1,30,0,1.951069,59,88
98,300,1,99,1,2,60,0,1.541265,59,89


In [24]:
'''
The hand coded sessions are done individually.'''
#in the s files, p2 is the confederate
df = organize_trial_by_trial_hand_code2('data/s14_gamedata_handcode.csv')
s14 = df
#s12_p1_g1_score,s12_p1_g2_score,s12_score_diff, s12_p1_g1_50,s12_p1_g2_50 = call_get_score(s12, 's')
s14.head()

s14_p1_counts, s14_p2_counts, s14_p1_probs, s14_p2_probs = handcode_calc_prob(s14)
s14_p1_probs

,p1_h1,p1_h2,p1_h3
game 1,0.0,88.659794,0.0
game 2,0.0,87.755102,0.0


In [25]:
'''The Hand Code Functions are called for s14'''
s14 = fill_in_prob(s14, 60,90,30,60,90,30,60,90,30,60,90,30)

s14_p1_g1, s14_p1_g2 = separate_df(s14, 's')

s14_p1_g1_score = s14_p1_g1['reward'].sum()
s14_p1_g2_score = s14_p1_g2['reward'].sum()

#s14 is added to the dictionary of dataframes
s_dict['s14'] = s14
s_dict['s14_p1_g1_score'] = s14_p1_g1_score
s_dict['s14_p1_g2_score'] = s14_p1_g2_score
s_dict['s14_p1_g1'] = s14_p1_g1
s_dict['s14_p1_g2'] = s14_p1_g2

/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [26]:
'''The %store command lets you pass variables between two different notebooks.
To read s_dict in another notebook:
%store -r s_dict'''

%store s_dict

Stored 's_dict' (dict)


# SMAB4 Control Data 

In [27]:
c_trial_by_trial = ['c02','c03','c05','c06','c09','c10','c11']
c_three = ['c12','c13','c14','c15','c16','c17','c18']

In [28]:
c_trial_by_trial_dict = imports(sessions = c_trial_by_trial, condition = 'c')

/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


c02
c03
c05
c06


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anacon

c09
c10
c11


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [29]:
c_three_dict = imports(sessions = c_three, condition = 'c', types = 'three')

c12


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


c13
c14
c15


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


c16
c17


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


c18


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [30]:
#combine into 1 dictionary
c_dict = {**c_trial_by_trial_dict, **c_three_dict}

In [31]:
'''The hand-coded sessions and c07(missing first 3 trials) are done separately.'''
#TO DO: resolve issue with c04


df = organize_trial_by_trial_hand_code2('data/c01_gamedata_handcode.csv')
c01 = df
#c02_p1_g1_score,c02_p1_g2_score,c02_score_diff, c02_p1_g1_50,c02_p1_g2_50 = call_get_score(c02, 'c')
c01.head()

#missing trials! excluded
df = organize_trial_by_trial_hand_code2('data/c04_gamedata_handcode.csv')
c04 = df
#c04_p1_g1_score,c04_p1_g2_score,c04_score_diff, c04_p1_g1_50,c04_p1_g2_50 = call_get_score(c04, 'c')
c04.head()

#missing first 3 trials
#in the s files, p2 is the confederate
df = organize_trial_by_trial('data/c07_gamedata.csv', types = 'default')
c07 = df
#call_get_score doesn't work for c07
c07_p1_g1_score,c07_p1_g2_score,c07_score_diff, c07_p1_g1_50,c07_p1_g2_50 = 78, 86, 8, 34, 42 
#c07.head()

In [32]:
c01

,game number,trial,player,arm,probability,reward,time,comment 1,comment 2
0,1,1,1,2.0,NaN,0.0,1900-01-01 00:02:57.169,NaT,
1,1,2,2,NaN,NaN,NaN,NaT,NaT,
2,1,3,1,1.0,NaN,1.0,1900-01-01 00:03:05.346,NaT,
3,1,4,2,NaN,NaN,NaN,NaT,NaT,
4,1,5,1,3.0,NaN,0.0,1900-01-01 00:03:12.216,NaT,
...,...,...,...,...,...,...,...,...,...
405,2,196,2,NaN,NaN,NaN,NaT,NaT,
406,2,197,1,1.0,NaN,1.0,1900-01-01 00:27:35.816,NaT,
407,2,198,2,NaN,NaN,NaN,NaT,NaT,
408,2,199,1,1.0,NaN,NaN,1900-01-01 00:27:42.082,NaT,


In [33]:
#c01 has 9 extra trials in game 1
#drop these trials
c01 = c01.drop(c01.index[200:210])
c01.reset_index(drop=True)

,game number,trial,player,arm,probability,reward,time,comment 1,comment 2
0,1,1,1,2.0,NaN,0.0,1900-01-01 00:02:57.169,NaT,
1,1,2,2,NaN,NaN,NaN,NaT,NaT,
2,1,3,1,1.0,NaN,1.0,1900-01-01 00:03:05.346,NaT,
3,1,4,2,NaN,NaN,NaN,NaT,NaT,
4,1,5,1,3.0,NaN,0.0,1900-01-01 00:03:12.216,NaT,
...,...,...,...,...,...,...,...,...,...
395,2,196,2,NaN,NaN,NaN,NaT,NaT,
396,2,197,1,1.0,NaN,1.0,1900-01-01 00:27:35.816,NaT,
397,2,198,2,NaN,NaN,NaN,NaT,NaT,
398,2,199,1,1.0,NaN,NaN,1900-01-01 00:27:42.082,NaT,


In [34]:
c01_p1_counts, c01_p2_counts, c01_p1_probs, c01_p2_probs = handcode_calc_prob(c01)
c01_p1_probs

,p1_h1,p1_h2,p1_h3
game 1,90.697674,58.823529,33.333333
game 2,84.745763,66.666667,9.090909


In [35]:
c04_p1_counts, c04_p2_counts, c04_p1_probs, c04_p2_probs = handcode_calc_prob(c04)
c04_p1_probs

,p1_h1,p1_h2,p1_h3
game 1,89.583333,54.545455,9.090909
game 2,87.719298,55.263158,5.555556


In [36]:
#c07_p1_counts, c07_p2_counts, c07_p1_probs, c07_p2_probs = handcode_calc_prob(c07)
#c07_p1_probs

In [37]:
c01 = fill_in_prob(c01, 90,60,30,90,60,30,90,60,30,90,60,30)
c04 = fill_in_prob(c04, 90,60,30,90,60,30,90,60,30,90,60,30)

In [38]:
c01_p1_g1, c01_p1_g2 = separate_df(c01, 'c')
c04_p1_g1, c04_p1_g2 = separate_df(c04, 'c')
c07_p1_g1, c07_p1_g2 = separate_df(c07, 'c')

/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [39]:
c01_p1_g1_score = c01_p1_g1['reward'].sum()
c01_p1_g2_score = c01_p1_g2['reward'].sum()
c04_p1_g1_score = c04_p1_g1['reward'].sum()
c04_p1_g2_score = c04_p1_g2['reward'].sum()

In [40]:
#add c01,c04,and c07 to the dictionary
c_dict['c07'] = c07
c_dict['c01'] = c01
c_dict['c04'] = c04

c_dict['c01_p1_g1_score'] = c01_p1_g1_score
c_dict['c01_p1_g2_score'] = c01_p1_g2_score
c_dict['c04_p1_g1_score'] = c04_p1_g1_score
c_dict['c04_p1_g2_score'] = c04_p1_g2_score
c_dict['c07_p1_g1_score'] = c07_p1_g1_score
c_dict['c07_p1_g2_score'] = c07_p1_g2_score

c_dict['c01_p1_g1'] = c01_p1_g1
c_dict['c01_p1_g2'] = c01_p1_g2
c_dict['c04_p1_g1'] = c04_p1_g1
c_dict['c04_p1_g2'] = c04_p1_g2
c_dict['c07_p1_g1'] = c07_p1_g1
c_dict['c07_p1_g2'] = c07_p1_g2

In [41]:
c_dict['c01_p1_g2']

,game number,trial,player,arm,probability,reward,time,comment 1,comment 2
0,2,1,1,1.0,90.0,1.0,1900-01-01 00:15:36.381,NaT,
1,2,3,1,2.0,60.0,0.0,1900-01-01 00:15:56.822,NaT,
2,2,5,1,3.0,30.0,0.0,1900-01-01 00:16:03.323,NaT,
3,2,7,1,1.0,90.0,1.0,1900-01-01 00:16:10.428,NaT,2Tou
4,2,9,1,1.0,90.0,1.0,1900-01-01 00:16:16.996,1900-01-01 00:16:18.269,
...,...,...,...,...,...,...,...,...,...
95,2,191,1,1.0,90.0,1.0,1900-01-01 00:27:15.308,NaT,
96,2,193,1,1.0,90.0,1.0,1900-01-01 00:27:22.479,NaT,
97,2,195,1,1.0,90.0,1.0,1900-01-01 00:27:29.516,NaT,
98,2,197,1,1.0,90.0,1.0,1900-01-01 00:27:35.816,NaT,


In [42]:
'''The %store command lets you pass variables between two different notebooks.
To read c_dict in another notebook:
%store -r c_dict'''

%store c_dict

Stored 'c_dict' (dict)


# SMAB3 social data

In [43]:
j_trial_by_trial = ['j01','j02','j03','j04','j05','j06','j07','j08']
j_three = ['j10','j11']

In [44]:
j_trial_by_trial_dict = imports(sessions = j_trial_by_trial, condition = 'j')

j01
j02
j03
j04
j05
j06
j07
j08


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python

In [45]:
j_three_dict = imports(sessions = j_three, condition = 'j', types='three')

/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


j10
j11


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [46]:
#combine dictionaries into one
j_dict = {**j_trial_by_trial_dict, **j_three_dict}

In [47]:
%store j_dict

Stored 'j_dict' (dict)


# SMAB4 social data

In [93]:
m_trial_by_trial = ['m05']
m_three = ['m08','m10','m11','m13','m14']

In [94]:
m_trial_by_trial_dict = imports(sessions = m_trial_by_trial, condition = 'm')

m05


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [104]:
m_three_dict = imports(sessions = m_three, condition = 'm', types = 'three')

m08
m10
m11
m13
m14


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python

In [106]:
m_three_dict['m10_p1_g1']

,type,game number,trial,player,arm,probability,reward,time,P1 score,P2 score
0,300,1,1,1,3,90,0,7.037642,1,0
1,300,1,3,1,3,90,0,4.402897,2,1
2,300,1,5,1,2,60,0,4.571818,2,2
3,300,1,7,1,1,30,0,7.626361,3,3
4,300,1,9,1,2,60,0,4.418891,4,4
5,300,1,11,1,3,90,0,4.324936,5,4
6,300,1,13,1,1,30,0,4.079052,5,5
7,300,1,15,1,3,90,0,3.914134,6,6
8,300,1,17,1,2,60,0,4.264966,7,7
9,300,1,19,1,1,30,0,4.045070,8,7


In [107]:
#combine into 1 dictionary
m_dict = {**m_trial_by_trial_dict, **m_three_dict}

In [108]:
m_dict['m08_p1_g1']

,type,game number,trial,player,arm,probability,reward,time,P1 score,P2 score
0,300,1,1,1,3,90,0,13.545530,1,0
1,300,1,3,1,3,90,0,5.116557,2,1
2,300,1,5,1,2,60,0,4.492857,2,2
3,300,1,7,1,1,30,0,3.699234,2,3
4,300,1,9,1,2,60,0,3.654255,3,3
5,300,1,11,1,1,30,0,3.729220,3,4
6,300,1,13,1,3,90,0,3.970104,4,5
7,300,1,15,1,1,30,0,3.458350,4,6
8,300,1,17,1,3,90,0,3.563298,5,7
9,300,1,19,1,3,90,0,3.459347,6,7


In [109]:
'''Hand coded sessions are done separately.'''

# # Multi Analysis

df = organize_trial_by_trial_hand_code2('data/m01_gamedata_handcode.csv')
m01 = df
#m01_p1_g1_score,m01_p1_g2_score,m01_p2_g1_score,m01_p2_g2_score,m01_p1_diff,m01_p2_diff,m01_p1_g1_50,m01_p1_g2_50,m01_p2_g1_50,m01_p2_g2_50 = call_get_score(m01, 'm')
m01.head()

df = organize_trial_by_trial_hand_code2('data/m03_gamedata_handcode.csv')
m03 = df
#m01_p1_g1_score,m01_p1_g2_score,m01_p2_g1_score,m01_p2_g2_score,m01_p1_diff,m01_p2_diff,m01_p1_g1_50,m01_p1_g2_50,m01_p2_g1_50,m01_p2_g2_50 = call_get_score(m01, 'm')
m03.head()

df = organize_trial_by_trial_hand_code2('data/m07_gamedata_handcode.csv')
m07 = df
#m01_p1_g1_score,m01_p1_g2_score,m01_p2_g1_score,m01_p2_g2_score,m01_p1_diff,m01_p2_diff,m01_p1_g1_50,m01_p1_g2_50,m01_p2_g1_50,m01_p2_g2_50 = call_get_score(m01, 'm')
m07.head()

#NOTE: m09 from LSL needs to be combined with hand coded data because the lsl csv file is missing the first 13 trials
df = organize_trial_by_trial_hand_code('data/m09_gamedata_handcode.csv')
m09_hand = df
#m01_p1_g1_score,m01_p1_g2_score,m01_p2_g1_score,m01_p2_g2_score,m01_p1_diff,m01_p2_diff,m01_p1_g1_50,m01_p1_g2_50,m01_p2_g1_50,m01_p2_g2_50 = call_get_score(m01, 'm')
m09_hand.head()

,game number,trial,player,arm,probability,reward,time,P1 score
0,1,1,1,1,,1,00:10.117,
1,1,2,2,3,,1,00:14.610,
2,1,3,1,2,,1,00:18.461,00:22.012
3,1,4,2,1,,0,00:27.408,
4,1,5,1,3,,0,00:31.720,


In [110]:
m01_p1_counts, m01_p2_counts, m01_p1_probs, m01_p2_probs = handcode_calc_prob(m01)
m03_p1_counts, m03_p2_counts, m03_p1_probs, m03_p2_probs = handcode_calc_prob(m03)
#m07_p1_counts, m07_p2_/counts, m07_p1_probs, m07_p2_probs = handcode_calc_prob(m07)

In [111]:
m01 = fill_in_prob(m01,90,60,30,60,30,90,30,60,90,60,90,30)
m03 = fill_in_prob(m03,30,90,60,90,30,60,90,30,60,30,90,60)

In [112]:
m01_p1_g1, m01_p1_g2, m01_p2_g1, m01_p2_g2 = separate_df(m01, 'm')
m03_p1_g1, m03_p1_g2, m03_p2_g1, m03_p2_g2 = separate_df(m03, 'm')
m07_p1_g1, m07_p1_g2, m07_p2_g1, m07_p2_g2 = separate_df(m07, 'm')

/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python

In [113]:
m01_p1_g1_score = m01_p1_g1['reward'].sum()
m01_p1_g2_score = m01_p1_g2['reward'].sum()
m01_p2_g1_score = m01_p2_g1['reward'].sum()
m01_p2_g2_score = m01_p2_g2['reward'].sum()

m03_p1_g1_score = m03_p1_g1['reward'].sum()
m03_p1_g2_score = m03_p1_g2['reward'].sum()
m03_p2_g1_score = m03_p2_g1['reward'].sum()
m03_p2_g2_score = m03_p2_g2['reward'].sum()

'''
m07_p1_g1_score = m07_p1_g1['reward'].sum()
m07_p1_g2_score = m07_p1_g2['reward'].sum()
m07_p2_g1_score = m07_p2_g1['reward'].sum()
m07_p2_g2_score = m07_p2_g2['reward'].sum()'''

"\nm07_p1_g1_score = m07_p1_g1['reward'].sum()\nm07_p1_g2_score = m07_p1_g2['reward'].sum()\nm07_p2_g1_score = m07_p2_g1['reward'].sum()\nm07_p2_g2_score = m07_p2_g2['reward'].sum()"

In [114]:
m01_p1_g1, m01_p1_g2, m01_p2_g1, m01_p2_g2 = separate_df(m01, 'm')
m03_p1_g1, m03_p1_g2, m03_p2_g1, m03_p2_g2 = separate_df(m03, 'm')

/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/baryosef/anaconda3/lib/python

In [115]:
m_dict['m01'] = m01
m_dict['m01_p1_g1_score'] = m01_p1_g1_score
m_dict['m01_p1_g2_score'] = m01_p1_g2_score
m_dict['m01_p2_g1_score'] = m01_p2_g1_score
m_dict['m01_p2_g2_score'] = m01_p2_g2_score
m_dict['m01_p1_g1'] = m01_p1_g1
m_dict['m01_p1_g2'] = m01_p1_g2
m_dict['m01_p2_g1'] = m01_p2_g1
m_dict['m01_p2_g2'] = m01_p2_g2


m_dict['m03'] = m03
m_dict['m03_p1_g1_score'] = m03_p1_g1_score
m_dict['m03_p1_g2_score'] = m03_p1_g2_score
m_dict['m03_p2_g1_score'] = m03_p2_g1_score
m_dict['m03_p2_g2_score'] = m03_p2_g2_score
m_dict['m03_p1_g1'] = m03_p1_g1
m_dict['m03_p1_g2'] = m03_p1_g2
m_dict['m03_p2_g1'] = m03_p2_g1
m_dict['m03_p2_g2'] = m03_p2_g2

'''m07 is not included in the rest of the analyses because the probability-hole mappings between pairs was not correct
m_dict['m07'] = m07
m_dict['m07_p1_g1_score'] = m07_p1_g1_score
m_dict['m07_p1_g2_score'] = m07_p1_g2_score
m_dict['m07_p2_g1_score'] = m07_p2_g1_score
m_dict['m07_p2_g2_score'] = m07_p2_g2_score
m_dict['m07_p1_g1'] = m07_p1_g1
m_dict['m07_p1_g2'] = m07_p1_g2
m_dict['m07_p2_g1'] = m07_p2_g1
m_dict['m07_p2_g2'] = m07_p2_g2'''

"m07 is not included in the rest of the analyses because the probability-hole mappings between pairs was not correct\nm_dict['m07'] = m07\nm_dict['m07_p1_g1_score'] = m07_p1_g1_score\nm_dict['m07_p1_g2_score'] = m07_p1_g2_score\nm_dict['m07_p2_g1_score'] = m07_p2_g1_score\nm_dict['m07_p2_g2_score'] = m07_p2_g2_score\nm_dict['m07_p1_g1'] = m07_p1_g1\nm_dict['m07_p1_g2'] = m07_p1_g2\nm_dict['m07_p2_g1'] = m07_p2_g1\nm_dict['m07_p2_g2'] = m07_p2_g2"

In [116]:
m_dict['m01_p1_g1_score']

67

In [117]:
pd.set_option('display.max_rows', None)

In [118]:
m_dict['m05_p1_g1']

,game number,trial,player,arm,probability,reward,time,P1 score,P2 score
0,1,1,1,3,30,1,12.377098,1,0
1,1,2,1,3,30,1,4.660777,2,1
2,1,3,1,1,90,1,4.084052,3,2
3,1,4,1,2,60,1,4.207993,4,2
4,1,5,1,1,90,1,3.530316,5,3
5,1,6,1,2,60,1,4.151021,6,3
6,1,7,1,2,60,1,4.534838,7,3
7,1,8,1,1,90,1,3.676247,8,4
8,1,9,1,2,60,1,3.450354,9,5
9,1,10,1,3,30,0,3.428366,9,6


In [119]:
%store m_dict

Stored 'm_dict' (dict)
